Download the GPT-2 tools repository, and the GPT-2 models, by using the download script which is part of the tools.


In [0]:
# clean
import os
os.chdir('/content')
!rm -fr gpt-2

In [0]:
# download the GPT-2 tools with my changes
![ -d '/content/gpt-2' ] || git clone https://github.com/enricoros/gpt-2/ /content/gpt-2
import os
os.chdir('/content/gpt-2')
# install the required libraries (assumes TF is already installed and configured)
!pip3 install -r requirements.txt > /dev/null
# download the pre-trained medium models
![ -d 'models/345M' ] || python download_model.py 345M
![ -d 'models/774M' ] || python download_model.py 774M

Test for GPU to be present:

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

We can use a cleaner output.

In [0]:
# reduce verbosity
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Generate an **unconditional** sample, where the only hyperparameters are the top_k threshold and the temperature. Everything else is left to the model.

In [0]:
!time python3 src/generate_unconditional_samples.py --model_name='774M' --nsamples=1 --top_k=40 --temperature=0.7 | tee unconditional_samples.txt 2> unconditional_samples.err

Generate a parametric **unconditional** sample:

In [0]:
#@title Generate a Story; configuration:
nsamples = 2 #@param {type:"slider", min:1, max:8, step:1, default:1}
top_k = 40 #@param {type:"slider", min:1, max:100, step:1, default:40}
temperature = 0.7 #@param {type:"number", default: 0.7}
length = 33 #@param {type:"slider", min:1, max:1024, step:16, default: 1024}

# run the parametric unconditional gen
!time python3 src/generate_unconditional_samples.py --model_name='774M' --nsamples={nsamples} --batch_size={nsamples} --top_k={top_k} --temperature={temperature} --length={length} | tee unconditional_samples.txt 2> unconditional_samples.err

Configure for South Park

In [0]:
!wget https://raw.githubusercontent.com/BobAdamsEE/SouthParkData/master/by-season/Season-16.csv --quiet --show-progress
!wget https://raw.githubusercontent.com/BobAdamsEE/SouthParkData/master/by-season/Season-17.csv --quiet --show-progress
!wget https://raw.githubusercontent.com/BobAdamsEE/SouthParkData/master/by-season/Season-18.csv --quiet --show-progress
!wget https://raw.githubusercontent.com/BobAdamsEE/SouthParkData/master/by-season/Season-19.csv --quiet --show-progress

In [0]:
!python3 -W ignore src/interactive_conditional_samples.py --model_name=345M --nsamples=2 --batch_size=2 --length=100 --top_k=40

In [0]:
!nvidia-smi